In [ ]:
##### Working Code ####
import notebookutils, csv
import os, json, time, pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from openai import AzureOpenAI


areas = ["Employment"]
# , "Information and Communication Technology"]
#        "Communications other than ICT",
#         "Design and Delivery of Programs and Services", "Procurement of goods, services and facilities",
#        "Transportation", "Built Environment"]

os.environ["AZURE_OPENAI_API_KEY"] = "xxxxxxxxxxxxxxxxxx"
os.environ["AZURE_OPENAI_ENDPOINT"] = "xxxxxxxxxxxxxxxxxxxxxxx"
os.environ["OPENAI_API_VERSION"] = "2024-02-01"




StatementMeta(, b590fb83-e2dd-4388-8283-07f01496f435, 5, Finished, Available, Finished)

In [2]:
# Working Code

def json_extract(obj, key):
    """Recursively fetch values from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    return values[0]


def extract_accessibility_area_plan_progress(sector, org, doc_type, processed_file_path, file_path):
    res = []
    """sec = []
    orgn = []
    d_t = []
    ar = []"""

    embeddings = AzureOpenAIEmbeddings(model="text-embedding-ada-002")
    client = AzureOpenAI()

    loader = PyPDFLoader(file_path)
    pages = loader.load_and_split()

    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(pages)
    faiss_index = FAISS.from_documents(texts, embeddings)

    for area in areas:
        query = f"What are the planned actions to address barriers for the ' {area} Area'?"
        docs = faiss_index.similarity_search(query, k=5)

        text = ""
        for doc in docs:
            text += doc.page_content

        if doc_type == "Plans":
            prompt = f"Extract the planned tasks and activities documented under the {area} section in the attached text :\n{text}"
        else:
            prompt = f"Extract the progress being made to implement the planned tasks and activities documented under the {area} section in the attached text :\n{text}"


        MESSAGES = [
            {"role": "system", "content": "You are an accessibility compliance analyst."},
            {"role": "user", "content": prompt},
        ]

        time.sleep(60)

        response = client.chat.completions.create(
            model="gpt-4-turbo-2024-04-09",
            temperature=0.0,
            top_p=0,
            messages=MESSAGES
        )

        parsed_data = json.loads(response.json())
        content = json_extract(parsed_data, "content")
        # sector, org, doc_type, area, content  
        # sec.append(sector)
        # orgn.append(org)
        # d_t.append(doc_type)
        # ar.append(area)
        res.append(content)
        time.sleep(60)
    create_or_append_json(sector, org, doc_type, area, res, processed_file_path)
    return res



def create_or_append_json(sector, org, doc_type, area, res, processed_file_path):

    for area_desc in res:
        new_data = {
            "sector": sector,
            "org": org,
            "doc_type": doc_type,
            "area": area,
            "area_desc": area_desc
            }

        # Check if file exists and has content
        if notebookutils.fs.exists("file:" + processed_file_path):
            # Read existing data
            with open(processed_file_path, 'r') as file:
                try:
                    data = json.load(file)
                    
                    # Handle different data structures
                    if isinstance(data, list):
                        data.append(new_data)
                    elif isinstance(data, dict):
                        if 'items' not in data:
                            data['items'] = []
                        data['items'].append(new_data)
                    else:
                        # If neither list nor dict, wrap existing data and new data in a list
                        data = [data, new_data]
                        
                except json.JSONDecodeError:
                    print("Error: File contains invalid JSON. Creating new file...")
                    data = {'items': [new_data]}
        else:
            # Create new file with initial data
            data = {'items': [new_data]}
        
        # Write data back to file
        with open(processed_file_path, 'w') as file:
            json.dump(data, file, indent=4)


StatementMeta(, b590fb83-e2dd-4388-8283-07f01496f435, 6, Finished, Available, Finished)

In [3]:
doc_type = "Plans"

StatementMeta(, a8e26522-4105-4fcd-9947-ca8f4d998957, 7, Finished, Available, Finished)

In [3]:
plans = notebookutils.fs.ls('file:/lakehouse/default/Files/Bronze/Plans')
progressreports = notebookutils.fs.ls('file:/lakehouse/default/Files/Bronze/ProgressReports')

if doc_type == "Plans":
    files = plans
    processed_file_path = "/lakehouse/default/Files/Silver/plans.json"
else:
    files = progressreports
    processed_file_path = "/lakehouse/default/Files/Silver/progressreports.json"


## Extract details from each accessibility sections
for file in files:
    file_split = file.name.rsplit("-")
    sector = file_split[1]
    org = file_split[2]
    response = extract_accessibility_area_plan_progress(sector, org, doc_type, processed_file_path, file.path.replace('file:', ''))



move_processed_files(files)


StatementMeta(, b590fb83-e2dd-4388-8283-07f01496f435, 7, Finished, Available, Finished)

In [ ]:
#### move files to processed folder
def move_processed_files (file_list):
    destination_path = "file:/lakehouse/default/Files/Bronze/ProcessedFiles"

    for file in file_list:
        # Move files from source to destination
        notebookutils.fs.mv(file.path, destination_path)

    # Verify if files were moved successfully
    print(f"Files moved from {source_path} to {destination_path}")

In [ ]:
# sector = "Telecom"
# org = "telecom2"
# area = "Employment"
# doc_type = "Progress Report"
# res = ["a","b","c"]
# res =  ["The planned tasks and activities documented under the Employment section in the text are as follows:\n\n1. **Regular Reviews of Accommodations Program**: Continue to conduct regular reviews of the accommodations program, in consultation with employees with disabilities, to identify opportunities to improve support. This task is ongoing.\n\n2. **Creation of an Employee Accessibility Hub**: Develop and continuously update an employee Accessibility Hub that will serve as a centralized source of accessibility-related resources. This includes training, policies, key contacts, and guidance on how to request accommodations and provide feedback. This task is categorized as short term.\n\n3. **Proactive Communication about Accessibility Policies and Offerings**: Enhance proactive communication regarding accessibility policies and offerings. This task is planned for the medium term.\n\n4. **Update Accessibility Training for Managers**: Update the accessibility training provided to managers to promote transparent conversations with employees about their disabilities, the accommodations they need, and how to better support them in the workplace. This task is also planned for the medium term.\n\n5. **Formalize Mentorship Programs**: Formalize mentorship programs to support employees with disabilities in their career development. This task is planned for the long term.", "'The planned tasks and activities documented under the Employment section in the text are as follows:\n\n1. **Regular Reviews of Accommodations Program**: Continue to conduct regular reviews of the accommodations program, in consultation with employees with disabilities, to identify opportunities to improve support. This task is ongoing.\n\n2. **Creation of an Employee Accessibility Hub**: Develop and continuously update an employee Accessibility Hub that will serve as a centralized source of accessibility-related resources. This includes training, policies, key contacts, and guidance on how to request accommodations and provide feedback. This task is categorized as short term.\n\n3. **Proactive Communication about Accessibility Policies and Offerings**: Enhance proactive communication regarding accessibility policies and offerings. This task is planned for the medium term.\n\n4. **Update Accessibility Training for Managers**: Update the accessibility training provided to managers to promote transparent conversations with employees about their disabilities, the accommodations they need, and how to better support them in the workplace. This task is also planned for the medium term.\n\n5. **Formalize Mentorship Programs**: Formalize mentorship programs to support employees with disabilities in their career development. This task is planned for the long term.","'The planned tasks and activities documented under the Employment section in the text are as follows:\n\n1. **Regular Reviews of Accommodations Program**: Continue to conduct regular reviews of the accommodations program, in consultation with employees with disabilities, to identify opportunities to improve support. This task is ongoing.\n\n2. **Creation of an Employee Accessibility Hub**: Develop and continuously update an employee Accessibility Hub that will serve as a centralized source of accessibility-related resources. This includes training, policies, key contacts, and guidance on how to request accommodations and provide feedback. This task is categorized as short term.\n\n3. **Proactive Communication about Accessibility Policies and Offerings**: Enhance proactive communication regarding accessibility policies and offerings. This task is planned for the medium term.\n\n4. **Update Accessibility Training for Managers**: Update the accessibility training provided to managers to promote transparent conversations with employees about their disabilities, the accommodations they need, and how to better support them in the workplace. This task is also planned for the medium term.\n\n5. **Formalize Mentorship Programs**: Formalize mentorship programs to support employees with disabilities in their career development. This task is planned for the long term.","'The planned tasks and activities documented under the Employment section in the text are as follows:\n\n1. **Regular Reviews of Accommodations Program**: Continue to conduct regular reviews of the accommodations program, in consultation with employees with disabilities, to identify opportunities to improve support. This task is ongoing.\n\n2. **Creation of an Employee Accessibility Hub**: Develop and continuously update an employee Accessibility Hub that will serve as a centralized source of accessibility-related resources. This includes training, policies, key contacts, and guidance on how to request accommodations and provide feedback. This task is categorized as short term.\n\n3. **Proactive Communication about Accessibility Policies and Offerings**: Enhance proactive communication regarding accessibility policies and offerings. This task is planned for the medium term.\n\n4. **Update Accessibility Training for Managers**: Update the accessibility training provided to managers to promote transparent conversations with employees about their disabilities, the accommodations they need, and how to better support them in the workplace. This task is also planned for the medium term.\n\n5. **Formalize Mentorship Programs**: Formalize mentorship programs to support employees with disabilities in their career development. This task is planned for the long term."]
# processed_file_path = "/lakehouse/default/Files/Silver/processed.json"
# create_or_append_json(sector, org, doc_type, area, res, processed_file_path)

StatementMeta(, b590fb83-e2dd-4388-8283-07f01496f435, 8, Finished, Available, Finished)